# **Multigrams: Full Workflow**
The multigram workflow mirrors the unigram workflow, but with two differences. First, instead of _creating_ a whitelist, you filter the multigram corpus _using_ a whitelist containing the top-_N_ unigrams. Second, the multigram workflow adds a **pivoting** step. Pivoting reorganizes the database so that it's easy to query year-ngram combinations. For instance, you can learn how many times the word "nuclear" appeared in 2011 by querying the key `[2011] nuclear`. This is useful for analyzing changes in word meanings over time.

## **Setup**
### Imports

In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
from stop_words import get_stop_words
from ngram_prep.ngram_filter.lemmatizer import SpacyLemmatizer
from ngram_prep.ngram_acquire import download_and_ingest_to_rocksdb
from ngram_prep.ngram_filter.config import PipelineConfig as FilterPipelineConfig
from ngram_prep.ngram_filter.config import FilterConfig
from ngram_prep.ngram_filter.pipeline.orchestrator import build_processed_db
from ngram_prep.ngram_pivot.config import PipelineConfig as PivotPipelineConfig
from ngram_prep.ngram_pivot.pipeline import run_pivot_pipeline
from ngram_prep.utilities.peek import db_head, db_peek, db_peek_prefix
from ngram_prep.utilities.notebook_logging import setup_notebook_logging

### Configure Paths

In [2]:
base_path = Path("/vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files")
raw_db = base_path / "5grams.db"
filtered_db = base_path / "5grams_processed.db"
pivoted_db = base_path / "5grams_pivoted.db"
filter_tmp_dir = base_path / "processing_tmp"
pivot_tmp_dir = base_path / "pivot_tmp"
whitelist_path = "/vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/1grams_processed.db/whitelist.txt"

## **Step 1: Download and Ingest**

In [3]:
setup_notebook_logging(
    workflow_name="multigrams_acquire",
    console=False
)

download_and_ingest_to_rocksdb(
    ngram_size=5,
    repo_release_id="20200217",
    repo_corpus_id="eng",
    db_path_stub="/vast/edk202/NLP_corpora/Google_Books/",
    file_range=(0, 19422),
    random_seed=76,
    workers=30,
    use_threads=False,
    ngram_type="tagged",
    overwrite_db=False,
    write_batch_size=1_000_000,
    open_type="write:packed24",
    compact_after_ingest=True
)

N-GRAM ACQUISITION PIPELINE
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Start Time: 2025-11-03 22:21:30

Download Configuration
════════════════════════════════════════════════════════════════════════════════════════════════════
Ngram repo:           https://books.storage.googleapis.com/?prefix=ngrams/books/20200217/eng/1-
DB path:              /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/1grams.db
File range:           0 to 23
Total files:          24
Files to get:         24
Skipping:             0
Download workers:     20
Batch size:           100,000
Ngram size:           1
Ngram type:           tagged
Overwrite DB:         True
DB Profile:           write:packed24

Download Progress
════════════════════════════════════════════════════════════════════════════════════════════════════


Files Processed: 100%|█████████████████████████████████████████████████████████| 24/24 [07:30<00:00]



Post-Ingestion Compaction
════════════════════════════════════════════════════════════════════════════════════════════════════
Initial DB size:         46.75 GB
Compaction completed in 0:03:11
Size before:             46.75 GB
Size after:              57.76 GB
Space saved:             -11.02 GB (-23.6%)

Processing complete!

Final Summary
════════════════════════════════════════════════════════════════════════════════════════════════════
Fully processed files:       24
Failed files:                0
Total entries written:       41,783,218
Write batches flushed:       24
Uncompressed data processed: 43.28 GB
Processing throughput:       68.86 MB/sec

End Time: 2025-11-03 22:32:13.598633
Total Runtime: 0:10:43.596855
Time per file: 0:00:26.816536
Files per hour: 134.2


## **Step 2: Filter and Normalize**
`config.py` contains generic defaults for the filtering pipeline. You can override these defaults by passing `FilterConfig` and `FilterPipelineConfig` objects to the `build_processed_db` function. As implemented here, we:
1. case-normalize the tokens
2. remove tokens containing non-alphanumeric text
3. remove stopwords using the `stop-words` package
4. remove tokens fewer than 3 characters
5. lemmatize the tokens using the `spaCy` package—first using a lookup table and falling back to rules when lookups fail
6. Create a whitelist of the top 6,000 most frequent words that pass a `pyenchant` spell-check and appear in all corpora from 1900–2019 (inclusive).

In [3]:
setup_notebook_logging(
    workflow_name="multigrams_filter",
    data_path=str(base_path),
    console=False
)

stop_set = set(get_stop_words("english"))
lemmatizer = SpacyLemmatizer(language="en")

filter_config = FilterConfig(
    stop_set=stop_set,
    lemma_gen=lemmatizer,
    whitelist_path=whitelist_path
)

pipeline_config = FilterPipelineConfig(
    src_db=raw_db,
    dst_db=filtered_db,
    tmp_dir=filter_tmp_dir,
    num_workers=30,
    use_smart_partitioning=True,
    num_initial_work_units=1000,
    cache_partitions=True,
    use_cached_partitions=True,
    samples_per_worker=500_000,
    work_unit_claim_order="random",
    flush_interval_s=5.0,
    mode="restart",
    progress_every_s=60.0,
    ingest_num_readers=10,
    ingest_batch_items=1_000_000,
    ingest_queue_size=3,
    compact_after_ingest=False
)

build_processed_db(pipeline_config, filter_config)

N-GRAM FILTER PIPELINE
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Configuration
════════════════════════════════════════════════════════════════════════════════════════════════════
Pipeline
Run mode: resume

Workers
Num Workers:        30
Initial work units: 1000
Profiles:           read=read:packed24, write=write:packed24
Flush interval:     5.0s

Files
Source: /vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams.db
Destination: /vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams_processed.db
Input whitelist: ..._corpora/Google_Books/20200217/eng/1gram_files/1grams_processed.db/whitelist.txt
  All tokens (min count: 1)
Output whitelist: None
Loading whitelist...
Loaded 6,000 tokens

Phase 1: Creating work units...
════════════════════════════════════════════════════════════════════════════════════════════════════
Resuming existing work units
Resuming: 0 completed, 0 processing, 0 pending

Pha

## **Step 3: Pivot to Yearly Indices**

In [ ]:
setup_notebook_logging(
    workflow_name="multigrams_pivot",
    data_path=str(base_path),
    console=False
)

pivot_config = PivotPipelineConfig(
    src_db=filtered_db,
    dst_db=pivoted_db,
    tmp_dir=pivot_tmp_dir,
    num_workers=30,
    num_initial_work_units=1000,
    cache_partitions=True,
    use_cached_partitions=True,
    samples_per_worker=500_000,
    work_unit_claim_order="random",
    flush_interval_s=15.0,
    progress_every_s=30.0,
    mode="restart",
    num_ingest_readers=1,
    ingest_buffer_shards=1,
    use_smart_partitioning=True,
    ingest_mode="direct_sst"
)

run_pivot_pipeline(pivot_config)


PARALLEL N-GRAM DATABASE PIVOT
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Start Time: 2025-11-04 15:16:56
Mode:       RESTART

Configuration
════════════════════════════════════════════════════════════════════════════════════════════════════
Source DB:            ...dk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams_processed.db
Target DB:            .../edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams_pivoted.db
Temp directory:       /vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/pivot_tmp

Parallelism
────────────────────────────────────────────────────────────────────────────────────────────────────
Workers:              30
Initial work units:   1000

Database Profiles
────────────────────────────────────────────────────────────────────────────────────────────────────
Reader profile:       read:packed24
Writer profile:       write:packed24
Ingest profile:       write:packed24

Flush

# Inspect Final Database
Here are three functions you can use to inspect the final database.

## `db_head`: First N records

In [3]:
db_head(str(pivoted_db), n=5)

First 5 key-value pairs:
────────────────────────────────────────────────────────────────────────────────────────────────────
[ 1] Key:   [1470] <UNK> <UNK> <UNK> <UNK> eng
     Value: 1 occurrences in 1 documents

[ 2] Key:   [1470] <UNK> <UNK> <UNK> atomic energy
     Value: 1 occurrences in 1 documents

[ 3] Key:   [1470] <UNK> <UNK> <UNK> eng <UNK>
     Value: 1 occurrences in 1 documents

[ 4] Key:   [1470] <UNK> <UNK> <UNK> much convenient
     Value: 1 occurrences in 1 documents

[ 5] Key:   [1470] <UNK> <UNK> <UNK> selection <UNK>
     Value: 1 occurrences in 1 documents



## `db_peek`: Records starting from a key

In [4]:
db_peek(str(pivoted_db), start_key="[2000] quick", n=5)

5 key-value pairs starting from 000007d0717569636b:
────────────────────────────────────────────────────────────────────────────────────────────────────
[ 1] Key:   [2000] quick <UNK> <UNK> <UNK> <UNK>
     Value: 8,774 occurrences in 8,333 documents

[ 2] Key:   [2000] quick <UNK> <UNK> <UNK> ability
     Value: 1 occurrences in 1 documents

[ 3] Key:   [2000] quick <UNK> <UNK> <UNK> able
     Value: 3 occurrences in 3 documents

[ 4] Key:   [2000] quick <UNK> <UNK> <UNK> accidental
     Value: 4 occurrences in 4 documents

[ 5] Key:   [2000] quick <UNK> <UNK> <UNK> accurate
     Value: 8 occurrences in 8 documents



## `db_peek_prefix`: Records matching a prefix

In [5]:
db_peek_prefix(str(pivoted_db), prefix="[2011] unite", n=5)

5 key-value pairs with prefix 000007db756e697465:
────────────────────────────────────────────────────────────────────────────────────────────────────
[ 1] Key:   [2011] unite <UNK> <UNK> <UNK> <UNK>
     Value: 24,615 occurrences in 19,900 documents

[ 2] Key:   [2011] unite <UNK> <UNK> <UNK> able
     Value: 12 occurrences in 12 documents

[ 3] Key:   [2011] unite <UNK> <UNK> <UNK> accomplish
     Value: 2 occurrences in 2 documents

[ 4] Key:   [2011] unite <UNK> <UNK> <UNK> accomplishment
     Value: 1 occurrences in 1 documents

[ 5] Key:   [2011] unite <UNK> <UNK> <UNK> accord
     Value: 5 occurrences in 5 documents

